## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2024-08-01 17:12:54.085393: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [28]:
y_df['Attrition'].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [5]:
# Create a list of at least 10 column names to use as X data
X = attrition_df[['Age', 'DistanceFromHome', 'Education', 'HourlyRate', 'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole']]


# Create X_df using your selected columns
X_df = pd.DataFrame(data=X)

# Show the data types for X_df
X_df.dtypes


Age                   int64
DistanceFromHome      int64
Education             int64
HourlyRate            int64
JobSatisfaction       int64
NumCompaniesWorked    int64
PercentSalaryHike     int64
TotalWorkingYears     int64
YearsAtCompany        int64
YearsInCurrentRole    int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# X_train = X_train.astype(np.float64)
# X_test = X_test.astype(np.float64)

In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train).astype('float32')
X_test_scaled = scaler.transform(X_test).astype('float32')


In [17]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = department_encoder.transform(y_train[['Department']]).astype('float32')
y_test_dept = department_encoder.transform(y_test[['Department']]).astype('float32')



In [18]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr = attrition_encoder.transform(y_train[['Attrition']]).astype('float32')
y_test_attr = attrition_encoder.transform(y_test[['Attrition']]).astype('float32')


In [19]:
y_train_dept.dtype


dtype('float32')

## Create, Compile, and Train the Model

In [20]:
# Find the number of columns in the X training data
num_columns = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu')(input_layer)
shared2 = layers.Dense(128, activation='relu')(shared1)

In [21]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared2)

# Create the output layer
department_output = layers.Dense(y_train_dept.shape[1], activation='softmax', name='department_output')(department_hidden)


In [22]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared2)

# Create the output layer
attrition_output = layers.Dense(y_train_attr.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)


In [35]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'} )

# Summarize the model
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │        704 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │      8,320 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │      4,128 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      4,128 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_7[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Train the model
model.fit(
    X_train,
    {
        'department_output': y_train_dept,
        'attrition_output': y_train_attr
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/100


28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - attrition_output_accuracy: 0.9298 - department_output_accuracy: 0.8886 - loss: 0.4533 - val_attrition_output_accuracy: 0.7647 - val_department_output_accuracy: 0.5882 - val_loss: 2.5669
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9614 - department_output_accuracy: 0.9002 - loss: 0.3658 - val_attrition_output_accuracy: 0.8100 - val_department_output_accuracy: 0.5882 - val_loss: 2.6968
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9477 - department_output_accuracy: 0.9266 - loss: 0.3429 - val_attrition_output_accuracy: 0.7964 - val_department_output_accuracy: 0.5611 - val_loss: 2.7172
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9647 - department_output_accuracy: 0.9268 - loss: 0.2869 - val_attrition_output_accuracy: 0.7738 - val_department_output_accuracy: 0.5656 - val_loss: 2.6336
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - at

In [37]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test), {
    'department_output': y_test_dept,
    'attrition_output': y_test_attr
})

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.7859 - department_output_accuracy: 0.5123 - loss: 4.8127 


In [38]:
# Print the accuracy for both department and attrition
pred_categories = ['attrition', 'department']
for i, cat in enumerate(pred_categories):
    print(f"{cat} accuracy: {results[i+1]}")

attrition accuracy: 0.7880434989929199
department accuracy: 0.5135869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I think accuracy is one of the best metrics to use. It usually doesn't tell the whole story and other metrics such as recall and precision can give you a better idea on how to improve the model but I think accuracy is usually a good starting point to see how effective your model is. In this case the accuracy for attrition is acceptable but the department accuracy could be improved on.

2. Softmax is used on the Department output layer because it is a multiclass classification and can provide equal probababilities for each class. The sigmoid function is used for the Attrition output layer because it is a binary classificaiton. 

3. We can collect more data to have a more equally balanced dataset. If collecting more data isn't possible then we can oversample or undersample to have equal y values to help with this model. We can look into using different activation functions on the hidden layers and even the output layer to see if performance improves. We could also train on more epochs.